# Extracting the coordinates of the centerlines for Svalbard glaciers

We use the OGGM model to compute the centerline in Svalard: https://docs.oggm.org/en/latest/flowlines.html

Glacier flowlines
Computing the flowlines is the first task to run after the definition of the local map and topography.

OGGM is a “flowline model”, which means that the glacier ice flow is assumed to happen along a representative “1.5D” flowline, as in the image below. “1.5D” here is used to emphasize that although glacier ice can flow only in one direction along the flowline, each point of the glacier has a geometrical width. This width means that flowline glaciers are able to match the observed area-elevation distribution of true glaciers, and can parametrize the changes in glacier width with thickness changes.

Geometrical centerlines: Centerline determination

Our algorithm is an implementation of the procedure described by Kienholz et al., (2014). Appart from some minor changes (mostly the choice of some parameters), we stay close to the original algorithm.

The basic idea is to find the terminus of the glacier (its lowest point) and a series of centerline “heads” (local elevation maxima). The centerlines are then computed with a least cost routing algorithm minimizing both (i) the total elevation gain and (ii) the distance to the glacier terminus

In [1]:
from oggm.utils import *
%matplotlib inline
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
import os
from oggm import cfg, utils, workflow, tasks, graphics
from oggm.core import inversion
cfg.initialize(logging_level='WORKFLOW')

import matplotlib.pyplot as plt
import calendar
import xarray as xr
import salem
import pandas as pd
import holoviews as hv
#hv.extension('bokeh')
import geoviews as gv
import geoviews.tile_sources as gts

# set default font size in plots
plt.rc('font', size=16)

cfg.initialize(logging_level='WORKFLOW')
cfg.PATHS['working_dir'] = utils.gettempdir(home='True')


2021-02-03 16:03:44: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2021-02-03 16:03:44: oggm.cfg: Multiprocessing switched ON according to the parameter file.
2021-02-03 16:03:44: oggm.cfg: Multiprocessing: using all available processors (N=8)
2021-02-03 16:03:46: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2021-02-03 16:03:46: oggm.cfg: Multiprocessing switched ON according to the parameter file.
2021-02-03 16:03:46: oggm.cfg: Multiprocessing: using all available processors (N=8)


In [2]:
cfg.PATHS['working_dir']

'/home/colili/tmp/OGGM/'

## Define the area for Svalbard (zone 07)

In [3]:
utils.get_rgi_dir(version='62')  # path to the data after download
svalbard = utils.get_rgi_region_file('07', version='62')  # Svalbard region

## Test on 2 glaciers

In [4]:
from oggm.utils import *
rgi_id = []
gdir1 = workflow.init_glacier_directories('RGI60-07.01458', from_prepro_level=3, prepro_border=80)[0]
gdir2 = workflow.init_glacier_directories('RGI60-07.01459', from_prepro_level=3, prepro_border=80)[0]
# centerline_tuna = write_centerlines_to_shape([gdir1, gdir2], path = '/home/colili/Documents/PhD/project_john/'+rgi_id)
# centerline_tuna = write_centerlines_to_shape([gdir1, gdir2], filesuffix = rgi_id, path = True)

2021-02-03 16:03:54: oggm.workflow: init_glacier_directories from prepro level 3 on 1 glaciers.
2021-02-03 16:03:54: oggm.workflow: Execute entity task gdir_from_prepro on 1 glaciers
2021-02-03 16:03:55: oggm.workflow: init_glacier_directories from prepro level 3 on 1 glaciers.
2021-02-03 16:03:55: oggm.workflow: Execute entity task gdir_from_prepro on 1 glaciers


## Access the coordinates of the centerline

In [11]:
centerline_data = get_main_glacier_segment(gdir1)
centerline_data['geometry'].coords.xy

(array('d', [17.531155229534072, 17.539779401482495, 17.53978988181899, 17.548410243243296, 17.557028067560708, 17.55703600248701, 17.557043935029114, 17.557051865188097, 17.557059792965017, 17.557067718360944, 17.557075641376944, 17.55708356201408, 17.557091480273417, 17.55709939615602, 17.557107309662943, 17.557115220795257, 17.55712312955402, 17.557131035940294, 17.557138939955138, 17.55714684159961, 17.557154740874775, 17.557162637781683, 17.557170532321393, 17.557178424494968, 17.55718631430346, 17.557194201747926, 17.548621276050014, 17.54863041840934, 17.548639558030537, 17.540071664658758, 17.540082057314628, 17.540092446859287, 17.531529580538148, 17.522969229344092, 17.522982123561597, 17.514425539501364, 17.514439682249513, 17.514453820767518, 17.50590225247046, 17.505917637649812, 17.50593301822943, 17.505948394211362, 17.505963765597656, 17.505979132390355, 17.505994494591505, 17.506009852203153, 17.50602520522733, 17.506040553666093, 17.506055897521467, 17.514623153717295

## Get centerlines in lonlat, return the main centerline, and convert the shape file to dataframe

In [13]:
def cust_get_centerline_lonlat(gdir):
    """Quick n dirty solution to write the centerlines as a shapefile"""

    cls = gdir.read_pickle('centerlines')
    olist = []
    for j, cl in enumerate(cls[::-1]):
        mm = 1 if j == 0 else 0
        gs = dict()
        gs['RGIID'] = gdir.rgi_id
        gs['LE_SEGMENT'] = np.rint(np.max(cl.dis_on_line) * gdir.grid.dx)
        gs['MAIN'] = mm
#         tra_func = partial(gdir.grid.ij_to_crs, crs=wgs84)
        tra_func = partial(gdir.grid.ij_to_crs, crs=wgs84)
        gs['geometry'] = shp_trafo(tra_func, cl.line)
        olist.append(gs)

    return olist

def get_main_glacier_segment(gdir):
    """
    Returns segment in gdir that is the main
    
    This is because there are several segments usually
    in the data but we only want the main segment since
    this is the centerline of the glacier
    
    does no error handling or correction if there
    is no main segment
    """
    for segment in cust_get_centerline_lonlat(gdir):
        if segment['MAIN'] == 1:
            return segment
        else:
            pass
        
def convert_shape_to_dataframe(gdir):
    """
    Converts the oggm output of the centerline finder function
    to lat and lon dataframe
    
    Also projects the lat and lon to UTM 33N
    """
#     centerline_data = cust_get_centerline_lonlat(gdir)[0] # returns only the first
#     centerline_data = cust_get_centerline_lonlat(gdir) # returns only the first
    centerline_data = get_main_glacier_segment(gdir)
    rgiid = centerline_data['RGIID']
#     x, y = centerline_data['geometry'].coords.xy
    p = pyproj.Proj(proj='utm', zone=33, ellps='WGS84')
    x, y = centerline_data['geometry'].coords.xy
    x, y = p(x, y)
    return pd.DataFrame({'rgiid':(rgiid,)*len(x), 'x':x, 'y':y})

convert_shape_to_dataframe(gdir2)

(array('d', [17.531155229534072, 17.539779401482495, 17.53978988181899, 17.548410243243296, 17.557028067560708, 17.55703600248701, 17.557043935029114, 17.557051865188097, 17.557059792965017, 17.557067718360944, 17.557075641376944, 17.55708356201408, 17.557091480273417, 17.55709939615602, 17.557107309662943, 17.557115220795257, 17.55712312955402, 17.557131035940294, 17.557138939955138, 17.55714684159961, 17.557154740874775, 17.557162637781683, 17.557170532321393, 17.557178424494968, 17.55718631430346, 17.557194201747926, 17.548621276050014, 17.54863041840934, 17.548639558030537, 17.540071664658758, 17.540082057314628, 17.540092446859287, 17.531529580538148, 17.522969229344092, 17.522982123561597, 17.514425539501364, 17.514439682249513, 17.514453820767518, 17.50590225247046, 17.505917637649812, 17.50593301822943, 17.505948394211362, 17.505963765597656, 17.505979132390355, 17.505994494591505, 17.506009852203153, 17.50602520522733, 17.506040553666093, 17.506055897521467, 17.514623153717295

## Apply the centerline method for all glaciers in Svalbard taking there RGI-ID and save df to csv

In [ ]:
gdf = gpd.read_file(svalbard)
list(gdf)

In [5]:
df = pd.DataFrame()
missing_glaciers = []

for rgi_id in gdf['RGIId']:   #[0:10]:
    
    dataholder = []
    surge = []
    BgnDate = []
    EndDate = []
    CenLon = []
    CenLat = []
    Area = []
    Zmin = []
    Zmax = []
    Zmed = []
    Slope = []
    Aspect=[]
    Lmax = []
    Status = []
    Connect = []
    Form = []
    TermType = []
    Linkages = []
    Name = []
    check_geom = []
    geometry = []
    
    try:
        surge.append(gdf[gdf.RGIId == rgi_id].Surging.values[0])
        BgnDate.append(gdf[gdf.RGIId == rgi_id].BgnDate.values[0])
        EndDate.append(gdf[gdf.RGIId == rgi_id].EndDate.values[0])
        CenLon.append(gdf[gdf.RGIId == rgi_id].CenLon.values[0])
        CenLat.append(gdf[gdf.RGIId == rgi_id].CenLat.values[0])
        Area.append(gdf[gdf.RGIId == rgi_id].Area.values[0])
        Zmin.append(gdf[gdf.RGIId == rgi_id].Zmin.values[0])
        Zmax.append(gdf[gdf.RGIId == rgi_id].Zmax.values[0])
        Zmed.append(gdf[gdf.RGIId == rgi_id].Zmed.values[0])
        Slope.append(gdf[gdf.RGIId == rgi_id].Slope.values[0])
        Aspect.append(gdf[gdf.RGIId == rgi_id].Aspect.values[0])
        Lmax.append(gdf[gdf.RGIId == rgi_id].Lmax.values[0])
        Status.append(gdf[gdf.RGIId == rgi_id].Status.values[0])
        Connect.append(gdf[gdf.RGIId == rgi_id].Connect.values[0])
        Form.append(gdf[gdf.RGIId == rgi_id].Form.values[0])
        TermType.append(gdf[gdf.RGIId == rgi_id].TermType.values[0])
        Linkages.append(gdf[gdf.RGIId == rgi_id].Linkages.values[0])
        Name.append(gdf[gdf.RGIId == rgi_id].Name.values[0])
        check_geom.append(gdf[gdf.RGIId == rgi_id].check_geom.values[0])
        geometry.append(gdf[gdf.RGIId == rgi_id].geometry.values[0])


        data_glaciers =  {'rgiid':rgi_id, 'Surge':surge,\
                          'BgnDate': BgnDate, 'EndDate':EndDate, 'CenLon':CenLon,\
                          'CenLat':CenLat, 'Area':Area, 'Zmin':Zmin, 'Zmax':Zmax,\
                          'Zmed':Zmed, 'Slope':Slope, 'Aspect':Aspect, 'Lmax':Lmax,\
                           'Status':Status, 'Connect':Connect, 'Form':Connect,\
                          'TermType':TermType, 'Linkages': Linkages, 'Name':Name, \
                          'check_geom': check_geom, 'geometry':check_geom}
        single_glacier = pd.DataFrame(data_glaciers)

        gdir = workflow.init_glacier_directories(rgi_id, from_prepro_level=3, prepro_border=80)[0]

        centerlinedf = convert_shape_to_dataframe(gdir)
        centerlinedf.set_index('rgiid', inplace=True)
        single_glacier.set_index('rgiid', inplace=True)
        combineddf = centerlinedf.join(single_glacier)
    #     combineddf = pd.concat()
        df = pd.concat([df, combineddf])
    except os.error:
        missing_glaciers.append(rgi_id)
        pass

df.to_csv('/home/colili/Documents/PhD/project_john/data_set/data_centerlines_svalbard/data_centerline_svalbard2.csv')   

NameError: name 'pd' is not defined